In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ensure current working directory is on path so Python can import your module
import os, sys
sys.path.append(os.getcwd())

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "Bocephus"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto', 'maxHeight': '400px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'whiteSpace': 'normal'},
    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    #FIXME Add in the code for your geolocation chart

    # Build dataframe from the datatable's current view (respecting filters/sorts)
    if viewData is None:
        dff = df.copy()
    else:
        dff = pd.DataFrame.from_records(viewData)

    # If table is empty, show an informative message
    if dff.shape[0] == 0:
        return html.Div("No data available to map.")

    # single-row selection -> convert list to index (if None, default to 0)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Defensive clamp
    row = max(0, min(row, dff.shape[0] - 1))

    # Attempt to pull latitude and longitude.
    # Prefer named columns if present, otherwise fall back to columns 13 and 14.
    lat = None
    lon = None

    possible_lat_cols = ['latitude', 'lat', 'y_coord', 'y', 'Latitude', 'LAT']
    possible_lon_cols = ['longitude', 'lon', 'x_coord', 'x', 'Longitude', 'LON']

    for c in possible_lat_cols:
        if c in dff.columns:
            try:
                lat = float(dff.iloc[row][c])
                break
            except Exception:
                lat = None
    for c in possible_lon_cols:
        if c in dff.columns:
            try:
                lon = float(dff.iloc[row][c])
                break
            except Exception:
                lon = None

    # Fallback to iloc positions 13 and 14 if lat/lon not found
    if (lat is None or pd.isna(lat)) and dff.shape[1] > 13:
        try:
            lat = float(dff.iloc[row, 13])
        except Exception:
            lat = None
    if (lon is None or pd.isna(lon)) and dff.shape[1] > 14:
        try:
            lon = float(dff.iloc[row, 14])
        except Exception:
            lon = None

    # If we still don't have valid coordinates, inform the user
    if lat is None or lon is None or pd.isna(lat) or pd.isna(lon):
        return html.Div("Selected row does not contain valid latitude/longitude coordinates.")

    # Get tooltip (breed) and popup (name) fields if present, else fall back to specific columns
    tooltip_text = dff.iloc[row]['breed'] if 'breed' in dff.columns else (str(dff.iloc[row,4]) if dff.shape[1] > 4 else "No breed")
    popup_text = dff.iloc[row]['name'] if 'name' in dff.columns else (str(dff.iloc[row,9]) if dff.shape[1] > 9 else "Unknown")

    # Build the leaflet map with a single marker for the selected row
    try:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48], zoom=10, children=[
               dl.TileLayer(id="base-layer-id"),
               dl.Marker(position=[lat, lon],
                  children=[
                  dl.Tooltip(tooltip_text),
                  dl.Popup([
                     html.H1("Animal Name"),
                    html.P(popup_text)
                 ])
              ])
           ])
        ]
    except Exception as e:
        return html.Div(f"Error creating map: {e}")
    

app.run_server(debug=True)


Read operation error: nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 68a2721d624a871cc161e59e, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 31580) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused')>]>
Dash app running on http://127.0.0.1:11285/
